In [1]:
import sys
import time
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import os
import wandb
from transformers import ViTMAEConfig

from custom_models.CustomViT import CustomViT
from custom_models.CustomViTMAE import CustomViTMAE
from transformers.models.vit_mae.modeling_vit_mae import ViTMAEModel
# from tem_dataloader import MultimodalDatasetPerTrajectory
from torch.utils.data import DataLoader

from d3rlpy.algos import CQL
from d3rlpy.dataset import Episode, MDPDataset, Transition
wandb.login() 

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: yadix. Use `wandb login --relogin` to force relogin


True

In [2]:
from custom_models.CustomViT import CustomViT
from custom_models.CustomViTMAE import CustomViTMAE
import torch
# call CustomViT
from transformers import AutoImageProcessor, ViTMAEForPreTraining, ViTMAEConfig
from PIL import Image

output_dir='/home/tyz/Desktop/11_777/camelmera/weights'

# Initialize a new CustomViTMAE model
model_name = "facebook/vit-mae-base"

vit_config = ViTMAEConfig.from_pretrained(model_name)
vit_config.output_hidden_states=True
vit_model = CustomViT.from_pretrained(model_name,config=vit_config)


model_name = "facebook/vit-mae-base"

config = ViTMAEConfig.from_pretrained(model_name)
config.output_hidden_states=True

# load from pretrained model and replace the original encoder with custom encoder
custom_model = CustomViTMAE.from_pretrained("facebook/vit-mae-base",config=config)
custom_model.vit = vit_model

# Load the state_dict from the saved model
state_dict = torch.load(f"{output_dir}/pytorch_model.bin")

# Apply the state_dict to the custom_model
custom_model.load_state_dict(state_dict)

Some weights of the model checkpoint at facebook/vit-mae-base were not used when initializing CustomViT: ['decoder.decoder_pos_embed', 'decoder.decoder_layers.5.output.dense.weight', 'decoder.decoder_layers.5.attention.attention.value.weight', 'decoder.decoder_layers.6.attention.attention.value.weight', 'decoder.decoder_layers.3.attention.attention.value.bias', 'decoder.decoder_layers.4.output.dense.weight', 'decoder.decoder_layers.3.output.dense.weight', 'decoder.decoder_layers.5.attention.attention.query.weight', 'decoder.decoder_layers.2.intermediate.dense.bias', 'decoder.decoder_layers.1.attention.attention.query.bias', 'decoder.decoder_layers.4.layernorm_before.bias', 'decoder.decoder_layers.4.attention.attention.key.bias', 'decoder.decoder_layers.5.layernorm_before.bias', 'decoder.decoder_layers.2.output.dense.bias', 'decoder.decoder_layers.6.attention.attention.key.bias', 'decoder.decoder_layers.4.intermediate.dense.weight', 'decoder.decoder_layers.7.output.dense.weight', 'decod

<All keys matched successfully>

In [14]:
from tem_dataloader import MultimodalDatasetPerTrajectory
environment_name = 'AmericanDinerExposure'
environemnt_directory = f'/media/tyz/3B6FFE7354FF3296/11_777/tartanairv2filtered/{environment_name}/Data_easy'
# my_dataset = MultimodalDatasetPerTrajectory(environemnt_directory)
for folder in os.listdir(environemnt_directory):
    trajectory_folder_path = os.path.join(environemnt_directory, folder)
    if not os.path.isdir(trajectory_folder_path):
        continue
    my_dataset = MultimodalDatasetPerTrajectory(trajectory_folder_path)
    train_dataloader = DataLoader(my_dataset, batch_size=32, shuffle=False)
    break

Processing folder: /media/tyz/3B6FFE7354FF3296/11_777/tartanairv2filtered/AmericanDinerExposure/Data_easy/P000


FileNotFoundError: [Errno 2] No such file or directory: '/media/tyz/3B6FFE7354FF3296/11_777/tartanairv2filtered/AmericanDinerExposure/Data_easy/P000/pose_lcam_front.txt'

In [9]:
for batch_idx, data in enumerate(train_dataloader):
    custom_model.cuda()
    custom_model.eval()
    pixel_values = data["pixel_values"].cuda()
    pixel_values1 = data["pixel_values1"].cuda()
    pixel_values2 = data["pixel_values2"].cuda()
    outputs = custom_model(pixel_values,pixel_values1,pixel_values2,noise=None)
    embedding = outputs.hidden_states[:,0,:]
    print(embedding.shape)
    break

torch.Size([32, 768])


In [15]:
print(embedding.shape)

torch.Size([32, 768])


In [ ]:
def get_action(trajectory_folder_path):
    pose = []
    pose_file_path = os.path.join(
            trajectory_folder_path, 'pose_lcam_front.txt')
    with open(pose_file_path) as f:
        lines = f.readlines()
        for line in lines:
            pose_list = line.split(" ")
            pose_list = [float(_) for _ in pose_list]
            pose.append(torch.Tensor(pose_list))
    return pose

In [23]:
'''
Args:
        observations (numpy.ndarray): N-D array. If the
            observation is a vector, the shape should be
            `(N, dim_observation)`. If the observations is an image, the shape
            should be `(N, C, H, W)`.
        actions (numpy.ndarray): N-D array. If the actions-space is
            continuous, the shape should be `(N, dim_action)`. If the
            action-space is discrete, the shape should be `(N,)`.
        rewards (numpy.ndarray): array of scalar rewards. The reward function
            should be defined as :math:`r_t = r(s_t, a_t)`.
        terminals (numpy.ndarray): array of binary terminal flags.
        episode_terminals (numpy.ndarray): array of binary episode terminal
            flags. The given data will be splitted based on this flag.
            This is useful if you want to specify the non-environment
            terminations (e.g. timeout). If ``None``, the episode terminations
            match the environment terminations.
        discrete_action (bool): flag to use the given actions as discrete
            action-space actions. If ``None``, the action type is automatically
            determined.
    '''
cql_dataset = MDPDataset(observations=embedding.cpu().detach().numpy(),\
                         actions=np.random.randn(32,7),\
                         rewards=np.zeros(32),\
                         terminals=np.zeros(32),\
                         episode_terminals=np.array([False]*embedding.shape[0]))

In [ ]:
from d3rlpy.algos import CQL

# setup CQL algorithm
cql = CQL(use_gpu=False)

# split train and test episodes
# train_episodes, test_episodes = train_test_split(cql_dataset, test_size=0.25)

# start training
cql.fit(cql_dataset,
        eval_episodes=None,
        n_epochs=1,
        scorers=None)